In [1]:
from pybaseball import team_ids, schedule_and_record, playerid_reverse_lookup
import pandas as pd
import numpy as np
import datetime
import time
import warnings
warnings.filterwarnings("ignore")

In [6]:
bat_data = pd.read_csv("Bat_Data_2022.csv",index_col = "Unnamed: 0")
pitch_data = pd.read_csv("Pitch_Data_2022.csv",index_col = "Unnamed: 0")
data = playerid_reverse_lookup(bat_data['mlbID'].unique(),key_type = "mlbam")
last_list = []
first_list = []
for x in range(len(data)):
    last_list.append(data.iloc[x]['name_last'].capitalize())
    first_list.append(data.iloc[x]['name_first'].capitalize())
data['name_last'] = last_list
data['name_first'] = first_list
data['Name'] = data['name_first'] + " " + data['name_last']
key_dict = {data['key_mlbam'][i]: data['Name'][i] for i in range(len(data))}
bat_data['mlbID'] = bat_data['mlbID'].apply(lambda x: key_dict[x])
bat_data['Name'] = bat_data['mlbID']

data = playerid_reverse_lookup(pitch_data['mlbID'].unique(),key_type = "mlbam")
last_list = []
first_list = []
for x in range(len(data)):
    last_list.append(data.iloc[x]['name_last'].capitalize())
    first_list.append(data.iloc[x]['name_first'].capitalize())
data['name_last'] = last_list
data['name_first'] = first_list
data['Name'] = data['name_first'] + " " + data['name_last']
key_dict = {data['key_mlbam'][i]: data['Name'][i] for i in range(len(data))}
pitch_data['mlbID'] = pitch_data['mlbID'].apply(lambda x: key_dict[x])
pitch_data['Name'] = pitch_data['mlbID']
bat_data['Team'] = bat_data.Lev + bat_data.Tm
pitch_data['Team'] = pitch_data.Lev + pitch_data.Tm
team_dict = {'Maj-NLMilwaukee':"MIL",
             'Maj-ALLos Angeles': "LAA",
             'Maj-NLAtlanta': "ATL",
             'Maj-NLArizona': "ARI",
             'Maj-NLNew York': "NYN",
             'Maj-ALHouston': "HOU",
             'Maj-NLCincinnati':"CIN",
             'Maj-NLSt. Louis': "SLN",
             'Maj-NLSan Diego': "SDN",
             'Maj-NLWashington': "WSN",
             'Maj-ALKansas City': "KCR",
             'Maj-ALCleveland': "CLE",
             'Maj-NLPittsburgh':"PIT",
             'Maj-NLChicago':"CHN", 
             'Maj-ALChicago':"CHA",
             'Maj-NLMiami':"MIA",
             'Maj-ALOakland':"OAK",
             'Maj-ALBoston':"BOS",
             'Maj-ALTampa Bay':"TBA",
             'Maj-ALMinnesota':"MIN",
             'Maj-ALDetroit':"DET",
             'Maj-NLSan Francisco':"SFN",
             'Maj-NLLos Angeles':"LAN",
             'Maj-ALToronto':"TOR",
             'Maj-NLColorado':"COL",
             'Maj-ALTexas':"TEX",
             'Maj-NLPhiladelphia':"PHI",
             'Maj-ALBaltimore':"BAL",
             'Maj-ALSeattle':"SEA",
             'Maj-ALNew York':"NYA"}
bat_data['Team'] = bat_data['Team'].apply(lambda x: team_dict[x])
pitch_data['Team'] = pitch_data['Team'].apply(lambda x: team_dict[x])
bat_data.drop(columns = ["Lev","Tm","mlbID"],inplace = True)
pitch_data.drop(columns = ["Lev","Tm","mlbID"],inplace = True)
schedule_df = pd.DataFrame()
for team in bat_data.Team.unique():
    s_r = schedule_and_record(2022,team)
    time.sleep(1)
    schedule_df = pd.concat([schedule_df,s_r])
date_list = []
for date in schedule_df['Date']:
    date = date.split(", ")[1] + ", 2022"
    date_list.append(date)
schedule_df['Date'] = date_list
bat_data['Home_Away'] = bat_data[bat_data.columns[4]]
pitch_data['Home_Away'] = pitch_data[pitch_data.columns[4]]
bat_important_stats = ['Name','Date','Opp','Team','Home_Away','PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'GDP', 'SB', 'CS']
pitch_important_stats = ['Name','Date','Opp','Team','Home_Away','GS','IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'SB', 'CS', 'PO', 'BF', 'GB/FB', 'LD', 'PU']
bat_data = bat_data[bat_important_stats]
pitch_data = pitch_data[pitch_important_stats]
schedule_df.drop(columns = ['R','RA','Inn','W-L','Rank','GB','Win','Loss','Save','Time','D/N','Attendance','cLI','Streak','Orig. Scheduled'],inplace = True)
record_list = []
for w_l in schedule_df['W/L']:
    if w_l == "L" or w_l == "L-wo":
        record_list.append(0)
    else:
        record_list.append(1)
schedule_df['W/L'] = record_list
clean_date_list = []
for date in schedule_df.Date:
    if len(date.split(")")) > 1:
        date = date.split(" (")[0] + date.split(")")[-1]
        clean_date_list.append(date)
    else:
        clean_date_list.append(date)
schedule_df['Date'] = clean_date_list
clean_abbrv_dict = {"NYM":"NYN",
                    "STL":"SLN",
                    "NYY":"NYA",
                    "TBR":"TBA",
                    "SFG":"SFN",
                    "CHC":"CHN",
                    "CHW":"CHA",
                    "SDP":"SDN",
                    "LAD":"LAN",}
for team in schedule_df.Tm.unique():
    try:
        clean_abbrv_dict[team]
    except:
        clean_abbrv_dict[team] = team
schedule_df['Tm'] = schedule_df['Tm'].apply(lambda x: clean_abbrv_dict[x])
schedule_df['Opp'] = schedule_df['Opp'].apply(lambda x: clean_abbrv_dict[x])
opp_list = []
for x in range(len(bat_data)):
    team = bat_data.iloc[x]['Team']
    date = bat_data.iloc[x]['Date']
    team_schedule = schedule_df[schedule_df['Tm'] == team]
    date_df = team_schedule[team_schedule['Date'] == date]
    opp_list.append(date_df.iloc[0]['Opp'])
bat_data['Opp'] = opp_list
opp_list = []
for x in range(len(pitch_data)):
    team = pitch_data.iloc[x]['Team']
    date = pitch_data.iloc[x]['Date']
    team_schedule = schedule_df[schedule_df['Tm'] == team]
    date_df = team_schedule[team_schedule['Date'] == date]
    opp_list.append(date_df.iloc[0]['Opp'])
pitch_data['Opp'] = opp_list
schedule_df.to_csv("Schedule_Record.csv")
bat_data.to_csv("Clean_Bat_2022.csv")
pitch_data.to_csv("Clean_Pitch_2022.csv")
bat_data.head()

http://www.baseball-reference.com/teams/MIL/2022-schedule-scores.shtml
http://www.baseball-reference.com/teams/LAA/2022-schedule-scores.shtml
http://www.baseball-reference.com/teams/ATL/2022-schedule-scores.shtml
http://www.baseball-reference.com/teams/ARI/2022-schedule-scores.shtml
http://www.baseball-reference.com/teams/NYN/2022-schedule-scores.shtml
http://www.baseball-reference.com/teams/HOU/2022-schedule-scores.shtml
http://www.baseball-reference.com/teams/CIN/2022-schedule-scores.shtml
http://www.baseball-reference.com/teams/SLN/2022-schedule-scores.shtml
http://www.baseball-reference.com/teams/SDN/2022-schedule-scores.shtml
http://www.baseball-reference.com/teams/WSN/2022-schedule-scores.shtml
http://www.baseball-reference.com/teams/KCR/2022-schedule-scores.shtml
http://www.baseball-reference.com/teams/CLE/2022-schedule-scores.shtml
http://www.baseball-reference.com/teams/PIT/2022-schedule-scores.shtml
http://www.baseball-reference.com/teams/CHN/2022-schedule-scores.shtml
http:/

,Name,Date,Opp,Team,Home_Away,PA,AB,R,H,2B,...,RBI,BB,IBB,SO,HBP,SH,SF,GDP,SB,CS
1,Willy Adames,"Apr 7, 2022",CHN,MIL,@,5,5,0,2,1,...,1,0,0,2,0,0,0,1,0,0
2,Jo Adell,"Apr 7, 2022",HOU,LAA,NaN,3,3,0,0,0,...,0,0,0,3,0,0,0,0,0,0
3,Ozzie Albies,"Apr 7, 2022",CIN,ATL,NaN,4,4,0,1,0,...,0,0,0,1,0,0,0,0,0,0
4,Sergio Alcántara,"Apr 7, 2022",SDN,ARI,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Pete Alonso,"Apr 7, 2022",WSN,NYN,@,5,4,1,2,0,...,0,0,0,0,1,0,0,1,0,0


In [7]:
pitch_data.head()

,Name,Date,Opp,Team,Home_Away,GS,IP,H,R,ER,...,IBB,GDP,SF,SB,CS,PO,BF,GB/FB,LD,PU
1,Víctor Arano,"Apr 7, 2022",NYN,WSN,NaN,0,1.0,0,0,0,...,0,1,0,0,0,0,2,1.00,0.00,0.00
2,Aaron Ashby,"Apr 7, 2022",CHN,MIL,@,0,1.2,3,1,1,...,0,1,0,0,0,0,8,0.67,0.17,0.17
3,Scott Barlow,"Apr 7, 2022",CLE,KCR,NaN,0,2.0,2,0,0,...,0,0,0,0,0,0,8,0.40,0.00,0.20
4,Shane Bieber,"Apr 7, 2022",KCR,CLE,@,1,4.2,2,1,0,...,0,0,0,0,0,0,18,0.21,0.14,0.21
5,Archie Bradley,"Apr 7, 2022",HOU,LAA,NaN,0,1.0,0,0,0,...,0,0,0,1,0,0,4,1.00,0.00,0.00
